Projekt 1
1. Napisz program, uczący się gry “papier, kamień, nożyce”. Jako algorytm uczący zastosuj łańcuch Markowa z macierzą przejść pomiędzy trzema stanami (Papier, Kamień, Nożyce). Nauka gry polega na korekcie macierzy przejść (prawdopodobieństwa warunkowe zmiany stanu),
2. Wartość wypłaty: 1 w przypadku wygranej, −1 w przypadku przegranej i 0 w przypadku remisu,
3. Przeprowadź ciąg kilkudziesięciu gier “kamień, papier, nożyce”. Sporząź wykres jak zmienia się stan kasy w każdym kroku gry.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
kpn = np.array(['KAMIEN', "PAPIER", 'NOZYCE'])

# pay +1 > wygranie , pay -1 > przegranie , pay -0 > remis
pay = 0
alpha = 0.1
count_of_game = 20
user_answer = ""
history_pay = list()
probability_matrix = np.array([
    #         Kamien Papier Nozyce
    # Kamien  [1 / 3, 1 / 3, 1 / 3],
    # Papier  [1 / 3, 1 / 3, 1 / 3],
    # Nozyce  [1 / 3, 1 / 3, 1 / 3]
    [1 / 3, 1 / 3, 1 / 3],
    [1 / 3, 1 / 3, 1 / 3],
    [1 / 3, 1 / 3, 1 / 3]
], dtype=np.double)


In [ ]:
previous = np.random.choice(kpn, p=[0.4, 0.3, 0.3])
print('Wylosowana poprzednia wartosc do startu gry: ', previous)


def adjust_probabilities(probability_matrix):
    # iterate over each row in the probability matrix
    for i in range(probability_matrix.shape[0]):
        row_sum = probability_matrix[i].sum()
        # check if the sum of the row is not equal to 1
        if not np.isclose(row_sum, 1):
            # find the index of the smallest value in the row
            min_index = np.argmin(probability_matrix[i])
            # adjust the smallest value to make the row sum to 1
            probability_matrix[i][min_index] += 1 - row_sum
    return probability_matrix


def update_probability_matrix(index, win_combination):
    global probability_matrix
    if win_combination == 'KAMIEN':
        probability_matrix[index][0] += (alpha - 0.05)
        probability_matrix[index][1] -= 0.025
        probability_matrix[index][2] -= 0.025
        probability_matrix = adjust_probabilities(probability_matrix)

    elif win_combination == 'PAPIER':
        probability_matrix[index][0] -= 0.025
        probability_matrix[index][1] += (alpha - 0.05)
        probability_matrix[index][2] -= 0.025
        probability_matrix = adjust_probabilities(probability_matrix)

    elif win_combination == 'NOZYCE':
        probability_matrix[index][0] -= 0.025
        probability_matrix[index][1] -= 0.025
        probability_matrix[index][2] += (alpha - 0.05)
        probability_matrix = adjust_probabilities(probability_matrix)


def check_who_win(user_answer, computer_choice, index):
    # Referencja do zmiennej pay
    global pay
    # Jak są rowne to znaczy istieje miejsce na jedną z wariancji: Kamien:Kamien , Papier:Papier, Nozyce:Nozyce
    if user_answer == computer_choice:
        print("\033[33mREMIS\033[0m")
        pay += 0
        history_pay.append(pay)
    elif user_answer == 'KAMIEN' and computer_choice == 'PAPIER':

        print("\033[31mCOMPUTER WIN WITH COMBINATION OF: ", computer_choice, "\033[0m")
        pay += -1
        history_pay.append(pay)
        # update_probability_matrix(index, computer_choice)

    elif user_answer == 'KAMIEN' and computer_choice == 'NOZYCE':
        print("\033[32mUSER WIN WITH COMBINATION OF: ", user_answer, "\033[0m")
        pay += 1
        history_pay.append(pay)
        update_probability_matrix(index, user_answer)

    elif user_answer == 'PAPIER' and computer_choice == 'KAMIEN':
        print("\033[32mUSER WIN WITH COMBINATION OF: ", user_answer, "\033[0m")
        pay += 1
        history_pay.append(pay)
        update_probability_matrix(index, user_answer)

    elif user_answer == 'PAPIER' and computer_choice == 'NOZYCE':
        print("\033[31mCOMPUTER WIN WITH COMBINATION OF: ", computer_choice, "\033[0m")
        pay += -1
        history_pay.append(pay)
        # update_probability_matrix(index, computer_choice)

    elif user_answer == 'NOZYCE' and computer_choice == 'KAMIEN':
        print("\033[31mCOMPUTER WIN WITH COMBINATION OF: ", computer_choice, "\033[0m")
        pay += -1
        history_pay.append(pay)
        # update_probability_matrix(index, computer_choice)

    elif user_answer == 'NOZYCE' and computer_choice == 'PAPIER':
        print("\033[32mUSER WIN WITH COMBINATION OF: ", user_answer, "\033[0m")
        pay += 1
        history_pay.append(pay)
        update_probability_matrix(index, user_answer)
    else:
        print('ERROR')


def probable_win_combination(choice):
    if choice == 'KAMIEN':
        return 'PAPIER'
    elif choice == 'PAPIER':
        return 'NOZYCE'
    elif choice == 'NOZYCE':
        return 'KAMIEN'


for i in range(count_of_game):

    user_answer = input("Podaj KAMIEN/PAPIER/NOZYCE: ")
    user_answer = user_answer.upper()
    # user_answer = np.random.choice(kpn, p=([1 / 3, 1 / 3, 1 / 3]))
    if previous == 'KAMIEN':
        choice = np.random.choice(kpn, p=probability_matrix[0])
        computer_choice = probable_win_combination(choice)
        check_who_win(user_answer, computer_choice, 0)
        print(probability_matrix)

    elif previous == 'PAPIER':
        choice = np.random.choice(kpn, p=probability_matrix[1])
        computer_choice = probable_win_combination(choice)
        check_who_win(user_answer, computer_choice, 1)
        print(probability_matrix)

    elif previous == 'NOZYCE':
        choice = np.random.choice(kpn, p=probability_matrix[2])
        computer_choice = probable_win_combination(choice)
        check_who_win(user_answer, computer_choice, 2)
        print(probability_matrix)

    previous = user_answer
    print('PREVIOUS', previous)
    print("COMPUTER CHOICE WAS: ", computer_choice)
    print('\n')
    plt.plot(history_pay, 'g--')
    plt.ylabel('PAY VALUE')
    plt.xlabel('GAMES VALUE')
    plt.show()
print('PAY: ', pay)